In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import warnings

warnings.filterwarnings("ignore")
plt.style.use("ggplot")
plt.rcParams.update({"font.size": 14, "axes.labelweight": "bold", "lines.linewidth": 2})

intertie_df = pd.read_csv('../../data/raw/intertie.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
intertie_df = intertie_df.rename_axis('date')
intertie_df = intertie_df.sort_values(by='date')
intertie_df = intertie_df.asfreq('H')
intertie_df = intertie_df.drop('Date - MST', axis=1)
intertie_df.head()


earliest_datetime = intertie_df.index.min()
latest_datetime = intertie_df.index.max()

print("Earliest datetime:", earliest_datetime)
print("Latest datetime:", latest_datetime)

Earliest datetime: 2020-12-20 00:00:00
Latest datetime: 2023-05-31 00:00:00


BC, MT and SK represents the flows between the respective regions and alberta. 

Imported - Out of the flows of BC, MT and SK, the flow of energy which are positive (imported) will go to the imported column   
Exported - Out of the flows of BC, MT and SK, the flow of energy which are positive (exported) will go to the exported column.

In the above table, a value which is positive means that it is imported and a value which is negative means that it is exported.

In [8]:
intertie_df['imported'] = intertie_df[['BC', 'MT', 'SK']].apply(lambda row: row[row > 0].sum(), axis=1)
intertie_df['exported'] = intertie_df[['BC', 'MT', 'SK']].apply(lambda row: row[row < 0].sum(), axis=1)
intertie_df['total_flow'] = intertie_df['WECC'] + intertie_df['SK'] #net flow between alberta and the interties 

In [9]:
intertie_df[intertie_df.isna().any(axis=1)]

,ATC SK Export,ATC SK Import,ATC WECC Export,ATC WECC Import,BC,MT,SK,TTC SK Export,TTC SK Import,TTC WECC Export,TTC WECC Import,WECC,imported,exported,total_flow
date,,,,,,,,,,,,,,,


In [10]:
intertie_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21409 entries, 2020-12-20 00:00:00 to 2023-05-31 00:00:00
Freq: H
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ATC SK Export    21409 non-null  int64  
 1   ATC SK Import    21409 non-null  int64  
 2   ATC WECC Export  21409 non-null  int64  
 3   ATC WECC Import  21409 non-null  int64  
 4   BC               21409 non-null  int64  
 5   MT               21409 non-null  float64
 6   SK               21409 non-null  int64  
 7   TTC SK Export    21409 non-null  int64  
 8   TTC SK Import    21409 non-null  int64  
 9   TTC WECC Export  21409 non-null  int64  
 10  TTC WECC Import  21409 non-null  int64  
 11  WECC             21409 non-null  float64
 12  imported         21409 non-null  float64
 13  exported         21409 non-null  float64
 14  total_flow       21409 non-null  float64
dtypes: float64(5), int64(10)
memory usage: 2.6 MB


In [11]:
column_mapping = {
    'ATC SK Export': 'atc_sk_export',
    'ATC SK Import': 'atc_sk_import',
    'ATC WECC Export': 'atc_wecc_export',
    'ATC WECC Import': 'atc_wecc_import',
    'TTC SK Export': 'ttc_sk_export',
    'TTC SK Import': 'ttc_sk_import',
    'TTC WECC Export': 'ttc_wecc_export',
    'TTC WECC Import': 'ttc_wecc_import',
    'BC': 'bc',
    'MT': 'mt',
    'SK': 'sk',
    'WECC': 'wecc'
}

intertie_df = intertie_df.rename(columns=column_mapping)



In [12]:
import os
folder_path = '../../data/processed'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the file inside the folder
file_path = os.path.join(folder_path, 'intertie.csv')
intertie_df.to_csv(file_path)

intertie_df.to_csv('../../data/processed/intertie.csv')
intertie_df.head()

,atc_sk_export,atc_sk_import,atc_wecc_export,atc_wecc_import,bc,mt,sk,ttc_sk_export,ttc_sk_import,ttc_wecc_export,ttc_wecc_import,wecc,imported,exported,total_flow
date,,,,,,,,,,,,,,,
2020-12-20 00:00:00,-153,153,-935,551,-50,199.552,0,-153,153,-1000,1110,149.552,199.552,-50.0,149.552
2020-12-20 01:00:00,-153,153,-935,551,0,199.616,0,-153,153,-1000,1110,199.616,199.616,0.0,199.616
2020-12-20 02:00:00,-153,153,-935,551,50,199.488,0,-153,153,-1000,1110,249.488,249.488,0.0,249.488
2020-12-20 03:00:00,-153,153,-935,551,50,199.552,0,-153,153,-1000,1110,249.552,249.552,0.0,249.552
2020-12-20 04:00:00,-153,153,-935,551,100,199.552,0,-153,153,-1000,1110,299.552,299.552,0.0,299.552


**The Available Transfer Capability (ATC)** is the maximum power that can be transmitted on an intertie (limit on power transmission as decided by AESO)
**TTC Total Transfer capability** is the maximum amount of power that can be transmitted on an intertie

BC - Actual flow between BC and Alberta     
WECC  - Actual flow between BC + MT and Alberta   
SK - Actual flow between SK and Alberta   
MT - Actual flow between Montana and Alberta   

BC, MT and SK represents the flows between the respective regions and alberta. 

imported - Out of the flows of BC, MT and SK, the flow of energy which are positive (imported) will go to the imported column   
exported - Out of the flows of BC, MT and SK, the flow of energy which are positive (exported) will go to the exported column.
total_flow - Net flow between intertie and alberta - Import + Export(-)

In the above table, a value which is positive means that it is imported and a value which is negative means that it is exported.